
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img
    src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png"
    alt="Databricks Learning"
  >
</div>

# 3.2 DEMO: Fine Grained Access Control using Dynamic Views and Partition Filtering [Recipient]

## Overview
This demonstration showcases how to consume data shared with two key data governance patterns:

- **Dynamic Views**: Row-level security based on recipient identity using `current_recipient()`
- **Partition Filtering**: Region-based data access control through table partitioning

**Provider Notebook:** Creates source data, implements one dynamic view example and one partition filtering example, then shares with recipients.

**Recipient Notebook (This Notebook):** Access shared views as the US partner recipient and observe how access controls work.

### Learning Objectives
By the end of this demo, you will be able to:
1. Mount and query shared data with dynamic access controls
2. Understand how recipient identity affects data visibility
3. Observe the difference between dynamic views and partition filtering

## Background

**Scenario:** You are the "US Partner" receiving customer order data from GlobalRetail Inc. The provider has sales data from multiple regions (United States, Europe, Asia Pacific) but you should only see data from the United States region through two different access control patterns.

## Setup

In [0]:
%run ./Includes/Demo-Setup-3.2

## Inspect Providers and Shares
Use the `SHOW PROVIDERS` and `SHOW SHARES` commands to see data that has been shared with your metastore

In [0]:
-- Show available providers
SHOW PROVIDERS;

In [0]:
-- Show shares in a provider
SHOW SHARES IN PROVIDER acme_corp;

## Example 1: Accessing Data via Dynamic Views

Dynamic views use the `current_recipient()` function to filter data based on who is accessing it. As the `us_partner` recipient, you will access the shared view and only see United States data.

In [0]:
-- Create catalog from the share
CREATE CATALOG IF NOT EXISTS shared_orders_dynamic
USING SHARE acme_corp.regional_orders_dynamic
COMMENT 'Orders shared via dynamic view';

In [0]:
-- Explore the shared schemas
SHOW SCHEMAS IN shared_orders_dynamic;

In [0]:
-- Use the catalog mounted from the share
USE CATALOG shared_orders_dynamic;

In [0]:
-- Show views in the shared schema
SHOW VIEWS IN sales;

In [0]:
-- Query the dynamic view
-- As 'us_partner', you'll only see United States orders
-- The view automatically filters based on current_recipient().region property
SELECT * 
FROM shared_orders_dynamic.sales.regional_orders_view
ORDER BY order_date;

**Key Observation:** 
- You only see orders from the United States region
- The filtering happens automatically based on your recipient identity (`us_partner`)
- Other partners accessing the same share see different data (their region only)

## Example 2: Accessing Data via Partition Filtering

Partition filtering shares specific partitions of a table. As `us_partner`, you will receive a share containing only the United States partition.

In [0]:
-- Show shares in the provider
SHOW SHARES IN PROVIDER acme_corp;

In [0]:
-- Create catalog from the partition-specific share
CREATE CATALOG IF NOT EXISTS us_orders_partition
USING SHARE acme_corp.regional_analytics_share
COMMENT 'United States orders via partition filtering';

In [0]:
-- Use our catalog
USE CATALOG us_orders_partition;

In [0]:
-- Explore the shared catalog
SHOW SCHEMAS;

In [0]:
-- Show the tables in the sales schema - we should see our partitioned table
SHOW TABLES IN sales;

### Step 3: Query the Partitioned Table

In [0]:
-- Query the partitioned table - you can only access the United States partition
SELECT * 
FROM us_orders_partition.sales.orders_by_region
ORDER BY order_date;

**Key Observation:**
- You only have access to the United States partition
- The data is physically separated at the storage level
- You cannot query or even see that other partitions (Europe, Asia Pacific) exist
- This provides storage-level isolation

## Summary

✅ **What we accomplished**

**Example 1 - Dynamic Views:**
- Mounted a shared catalog containing a dynamic view
- Queried the view and automatically received only United States data
- The `current_recipient()` function filtered rows based on our identity
- Other partners access the same share but see different data

**Example 2 - Partition Filtering:**
- Mounted a partition-specific share containing only United States data
- Queried the partitioned table with storage-level isolation
- Cannot access or see other partitions (Europe, Asia Pacific)

**Comparison:**
| Pattern | What You See | How It Works | Performance |
|---------|--------------|--------------|-------------|
| Dynamic Views | Only US rows | View filters at query time using `current_recipient()` | Good for moderate data |
| Partition Filtering | Only US partition | Share contains only US partition | Excellent (storage-level) |

**Key Takeaways:**
- Both patterns successfully restrict you to United States data only
- Dynamic views provide flexible logic-based filtering (however these require compute at the provider's end)
- Partition filtering provides storage-level isolation with better performance

&copy; 2025 Databricks, Inc. All rights reserved.